# Price Visualisation

Interactive candlestick charts with dynamic zoom-based resolution switching.

## Data generation

```bash
# Single week (quick test)
./build/qrsdp_run --seed 42 --days 5 --seconds 23400

# Full year (252 trading days, ~580M events, ~6.8 GB)
./build/qrsdp_run --seed 42 --days 252 --seconds 23400
```

In [1]:
import sys
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
from IPython.display import display

import qrsdp_reader as reader
import book_replay as replay
import ohlc

In [2]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")
SINGLE_DAY_FILE = None  # auto-detect first day from manifest

manifest = reader.load_manifest(RUN_DIR)
sessions = manifest["sessions"]
print(f"Run directory: {RUN_DIR}")
print(f"Sessions: {len(sessions)}, dates: {sessions[0]['date']} .. {sessions[-1]['date']}")
print(f"Seed: {manifest['base_seed']}, strategy: {manifest['seed_strategy']}")

Run directory: ../output/run_42
Sessions: 5, dates: 2026-01-02 .. 2026-01-08
Seed: 42, strategy: sequential


## Single-Day Deep Dive

Load one day, replay the order book, and show a zoomable candlestick chart
that dynamically switches resolution as you zoom in/out.

In [3]:
day_file = SINGLE_DAY_FILE or (RUN_DIR / sessions[0]["file"])
header = reader.read_header(day_file)
events = reader.read_day(day_file)
print(f"Loaded {len(events):,} events from {day_file.name}")
print(f"p0 = {header['p0_ticks']}, tick_size = {header['tick_size']}, session = {header['session_seconds']}s")

Loaded 2,262,506 events from 2026-01-02.qrsdp
p0 = 10000, tick_size = 100, session = 23400s


In [4]:
book = replay.replay_book(
    events,
    p0_ticks=header["p0_ticks"],
    levels_per_side=header["levels_per_side"],
    initial_spread_ticks=header["initial_spread_ticks"],
    initial_depth=header["initial_depth"],
)
print(f"Mid range: {book['mid_ticks'].min():.1f} – {book['mid_ticks'].max():.1f}")
print(f"Spread range: {book['spread_ticks'].min()} – {book['spread_ticks'].max()}")

Mid range: 9753.0 – 10008.0
Spread range: 2 – 2


In [5]:
bars = ohlc.multi_resolution_ohlc(book["ts_ns"], book["mid_ticks"])
for label, df in bars.items():
    print(f"  {label:>4s}: {len(df):>6,} bars")

    1s: 23,400 bars
   10s:  2,340 bars
  1min:    390 bars
  5min:     78 bars


In [6]:
# --- Zoomable candlestick chart ---
initial_key, initial_bars = ohlc.select_resolution(bars, header["session_seconds"])

fig = go.FigureWidget(
    data=[
        go.Candlestick(
            x=initial_bars["time_s"],
            open=initial_bars["open"],
            high=initial_bars["high"],
            low=initial_bars["low"],
            close=initial_bars["close"],
            name="Mid-price",
        )
    ],
    layout=go.Layout(
        title=f"Mid-Price — {sessions[0]['date']} (resolution: {initial_key})",
        xaxis=dict(
            title="Time (seconds from session start)",
            rangeslider=dict(visible=True),
        ),
        yaxis=dict(title="Price (ticks)"),
        height=550,
        template="plotly_white",
    ),
)


def _on_zoom(layout, xrange):
    """Switch candlestick resolution when the user zooms."""
    if xrange is None or len(xrange) < 2:
        return
    visible_seconds = abs(xrange[1] - xrange[0])
    key, best = ohlc.select_resolution(bars, visible_seconds)
    with fig.batch_update():
        fig.data[0].x = best["time_s"]
        fig.data[0].open = best["open"]
        fig.data[0].high = best["high"]
        fig.data[0].low = best["low"]
        fig.data[0].close = best["close"]
        fig.layout.title.text = (
            f"Mid-Price — {sessions[0]['date']} (resolution: {key})"
        )


fig.layout.on_change(_on_zoom, "xaxis.range")
display(fig)

FigureWidget({
    'data': [{'close': {'bdata': ('AAAAAAB8w0AAAAAAAHzDQAAAAACAds' ... 'AAAAAiw0AAAAAAABvDQAAAAACAGsNA'),
                        'dtype': 'f8'},
              'high': {'bdata': ('AAAAAACJw0AAAAAAgH/DQAAAAAAAfM' ... 'AAAAApw0AAAAAAACjDQAAAAAAAHcNA'),
                       'dtype': 'f8'},
              'low': {'bdata': ('AAAAAAB8w0AAAAAAAHfDQAAAAACAbM' ... 'AAAAAgw0AAAAAAgBrDQAAAAACAFcNA'),
                      'dtype': 'f8'},
              'name': 'Mid-price',
              'open': {'bdata': ('AAAAAACIw0AAAAAAAHzDQAAAAAAAfM' ... 'AAAIAlw0AAAAAAACLDQAAAAAAAG8NA'),
                       'dtype': 'f8'},
              'type': 'candlestick',
              'uid': '522321ce-a6be-48c5-b336-22c8287666ad',
              'x': {'bdata': ('PEpLA0Rjaz+mAaKxDcByQNMA0dgGwI' ... 'jGNgD51UAHiMY2AETWQAeIxjYAj9ZA'),
                    'dtype': 'f8'}}],
    'layout': {'height': 550,
               'template': '...',
               'title': {'text': 'Mid-Price — 2026-01-02 (resolution: 5m

### Bid-Ask Spread

In [7]:
# Down-sample spread for plotting (every 1000th event)
stride = max(1, len(book["ts_ns"]) // 20000)
t_s = book["ts_ns"][::stride] / 1e9
spread = book["spread_ticks"][::stride]

fig_spread = go.Figure(
    data=go.Scatter(x=t_s, y=spread, mode="lines", name="Spread"),
    layout=go.Layout(
        title=f"Bid-Ask Spread — {sessions[0]['date']}",
        xaxis=dict(title="Time (s)", rangeslider=dict(visible=True)),
        yaxis=dict(title="Spread (ticks)"),
        height=350,
        template="plotly_white",
    ),
)
fig_spread.show()

## Multi-Day Overview

Iterate all days, compute 5-min OHLC per day, and concatenate for a full-run
candlestick chart.

In [8]:
import pandas as pd

all_bars = []
for date, day_events in reader.iter_days(RUN_DIR):
    hdr = reader.read_header(RUN_DIR / [s["file"] for s in sessions if s["date"] == date][0])
    bk = replay.replay_book(
        day_events,
        p0_ticks=hdr["p0_ticks"],
        levels_per_side=hdr["levels_per_side"],
        initial_spread_ticks=hdr["initial_spread_ticks"],
        initial_depth=hdr["initial_depth"],
    )
    day_bars = ohlc.compute_ohlc(bk["ts_ns"], bk["mid_ticks"], ohlc.RESOLUTIONS["5min"])
    day_bars["date"] = date
    all_bars.append(day_bars)
    print(f"  {date}: {len(day_events):>10,} events -> {len(day_bars)} bars, "
          f"close={day_bars['close'].iloc[-1]:.1f}")

combined = pd.concat(all_bars, ignore_index=True)
combined["bar_idx"] = range(len(combined))
print(f"\nTotal: {len(combined):,} bars across {len(all_bars)} days")

  2026-01-02:  2,262,506 events -> 78 bars, close=9781.0
  2026-01-05:  2,260,762 events -> 78 bars, close=9953.0
  2026-01-06:  2,261,351 events -> 78 bars, close=9932.0
  2026-01-07:  2,263,572 events -> 78 bars, close=9913.0
  2026-01-08:  2,262,111 events -> 78 bars, close=9768.0

Total: 390 bars across 5 days


In [9]:
fig_multi = go.Figure(
    data=go.Candlestick(
        x=combined["bar_idx"],
        open=combined["open"],
        high=combined["high"],
        low=combined["low"],
        close=combined["close"],
        name="Mid-price (5min bars)",
    ),
    layout=go.Layout(
        title="Multi-Day Mid-Price (5-min OHLC)",
        xaxis=dict(
            title="Bar index (across all days)",
            rangeslider=dict(visible=True),
        ),
        yaxis=dict(title="Price (ticks)"),
        height=550,
        template="plotly_white",
    ),
)
fig_multi.show()